In [1]:
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image
import matplotlib.animation as animation
from IPython.display import HTML
import dotenv

%matplotlib notebook

In [2]:
root = dotenv.dotenv_values()['DATA_ROOT']

In [3]:
train_image_root = f'{root}/train_images'
train_data_path = f'{root}/train.csv'
scan_path = f'{root}/train_series_meta.csv'
injuries_path = f'{root}/image_level_labels.csv'
tag_path = f'{root}/train_dicom_tags.parquet'

In [4]:
train = pd.read_csv(train_data_path)
scans = pd.read_csv(scan_path)
injuries = pd.read_csv(injuries_path)
tags = pd.read_parquet(tag_path)

## CSV Analysis
### Tags

In [5]:
interesting_tags = []
for column in tags.columns:
    cnt = tags[column].nunique()
    print(f'Number of unique values in {column}: {cnt}')
    if 1 < cnt <= 20:
        interesting_tags.append(column)
for tag in interesting_tags:
    g = tags[tag].value_counts()
    print(f'Frequencies for {tag}: {[(i, g[i]) for i in g.index]}')
# Might analyze PatientPosition, SliceThickness for multi-image analysis

Number of unique values in BitsAllocated: 1
Number of unique values in BitsStored: 3
Number of unique values in Columns: 136
Number of unique values in ContentDate: 2
Number of unique values in ContentTime: 1508830
Number of unique values in FrameOfReferenceUID: 3147
Number of unique values in HighBit: 3
Number of unique values in ImageOrientationPatient: 173
Number of unique values in ImagePositionPatient: 1426166
Number of unique values in InstanceNumber: 1971
Number of unique values in KVP: 10
Number of unique values in PatientID: 3147
Number of unique values in PatientPosition: 2
Number of unique values in PhotometricInterpretation: 1
Number of unique values in PixelRepresentation: 2
Number of unique values in PixelSpacing: 807
Number of unique values in RescaleIntercept: 4
Number of unique values in RescaleSlope: 1
Number of unique values in Rows: 6
Number of unique values in SOPInstanceUID: 1210859
Number of unique values in SamplesPerPixel: 1
Number of unique values in SeriesIns

### Scans

In [43]:
scans_per_person = scans.groupby(['patient_id']).count()['series_id'].value_counts()
ax = plt.subplots()[1]
scans_per_person.plot.bar(ax=ax)
ax.set_xlabel('# Scans')
ax.set_ylabel("Count")
ax.set_title("Number of scans per person")
plt.show()

<IPython.core.display.Javascript object>

Finding the number of images in a scan

In [ ]:
num_images_per_scan = scans.loc[::20].apply(lambda row: len(os.listdir(f'{train_image_root}/{row["patient_id"].astype(np.uint16)}/{row["series_id"].astype(np.uint16)}')), axis=1)

In [45]:
(ax, cax, bax) = plt.subplots(1, 3, figsize=(10, 8))[1]
num_images_per_scan.hist(ax=ax)
ax.set_xlabel('Number of Images')
ax.set_ylabel('Count')
ax.set_title('Num Image per scan')

num_images_per_scan.plot(ax=bax, kind='box')
bax.set_ylabel('Number of Images')
bax.set_title('Num Image per scan')

num_images_per_scan.loc[num_images_per_scan <= 400].hist(ax=cax)
cax.set_xlabel('Number of Images')
cax.set_ylabel('Count')
cax.set_title('Num Image per scan (Truncated)')
plt.show()

/tmp/ipykernel_425/3072388478.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  (ax, cax, bax) = plt.subplots(1, 3, figsize=(10, 8))[1]


<IPython.core.display.Javascript object>

### Injuries

In [9]:
injuries['injury_name'].value_counts()

injury_name
Active_Extravasation    6370
Bowel                   5659
Name: count, dtype: int64

In [51]:
# which slices generally show the injuries
ax = plt.subplots()[1]
injuries['instance_number'].hist(ax=ax)
plt.show()

<IPython.core.display.Javascript object>

### Train

In [10]:
train.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1


In [11]:
fig, ax = plt.subplots(2, 3, figsize=(20, 10))
train[['bowel_healthy', 'bowel_injury']].sum().plot.bar(ax=ax[0][0])
train[['extravasation_healthy', 'extravasation_injury']].sum().plot.bar(ax=ax[0][1])
train[['kidney_healthy', 'kidney_low', 'kidney_high']].sum().plot.bar(ax=ax[0][2])
train[['liver_healthy', 'liver_low', 'liver_high']].sum().plot.bar(ax=ax[1][0])
train[['spleen_healthy', 'spleen_low', 'spleen_high']].sum().plot.bar(ax=ax[1][1])
train[['any_injury']].value_counts().rename({0: 'no_injury', 1: 'present_injury'}).plot.bar(ax=ax[1][2])

<IPython.core.display.Javascript object>

<AxesSubplot:xlabel='any_injury'>

### Extract Patient Categories

In [12]:
bowel_images = injuries[injuries['injury_name']=='Bowel']
fluid_images = injuries[injuries['injury_name']=='Active_Extravasation']
def image_frame_to_path(images):
    return train_image_root + '/' + images['patient_id'].astype(str) + '/' + images['series_id'].astype(str) + '/' + images['instance_number'].astype(str) + '.dicom'
bowel_image_paths = image_frame_to_path(bowel_images)
fluid_image_paths = image_frame_to_path(bowel_images)

In [53]:
# Number of images for each scan
bowel_images.groupby(['patient_id', 'series_id']).count()

instance_number  injury_name
patient_id series_id                              
33         55570                   35           35
43         24055                   50           50
820        11921                   28           28
           38809                   34           34
4093       5176                    98           98
...                               ...          ...
63665      13734                   28           28
           19848                   29           29
64194      25349                   58           58
           34232                   16           16
65456      40781                   16           16

[95 rows x 2 columns]

## Open dicom images

In [41]:
artists = []
anim = None
def animate_scan(path, frame_split=10, frame_rate=100):
    global artists, anim
    image_paths = sorted(os.listdir(path), key=lambda x: int(x.split('.')[0]))[::frame_split]
    
    fig, ax = plt.subplots()
    artists = []
    for image_path in image_paths:
        ds = pydicom.dcmread(f'{path}/{image_path}')
        container = ax.imshow(ds.pixel_array)
        artists.append([container])
        
    anim = animation.ArtistAnimation(fig=fig, artists=artists, interval=frame_rate)
    HTML(anim.to_jshtml())

In [42]:
animate_scan(f'{train_image_root}/10004/21057')

<IPython.core.display.Javascript object>

### Telling Frames in Unhealthy Scans